In [ ]:
pip install pandas

In [ ]:
import pandas as pd
import csv

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
arquivos = {
    '2019': 'notebooks/data_old/func2019_completo.csv',
    '2020': 'notebooks/data_old/func2020_completo.csv',
    '2021': 'notebooks/data_old/func2021_completo.csv'
}

lista_dataframes = []

for ano, arquivo in arquivos.items():
    try:
        # quoting=csv.QUOTE_NONE: Necessário pois o arquivo original possui aspas envolvendo as linhas inteiras
        df = pd.read_csv(arquivo, sep=',', quoting=csv.QUOTE_NONE, engine='python', on_bad_lines='skip')

        # Limpeza de aspas residuais na primeira e última coluna causadas pelo quoting=QUOTE_NONE
        col_primeira = df.columns[0]
        col_ultima = df.columns[-1]

        df[col_primeira] = df[col_primeira].str.replace('"', '')
        df[col_ultima] = df[col_ultima].str.replace('"', '')
        df.columns = df.columns.str.replace('"', '')

        df.to_csv(f'/data/dados_servidores_{ano}.csv', index=False)
        lista_dataframes.append(df)

    except Exception as e:
        print(f"Erro ao ler os dados do ano {ano}: {e}")